# Применение TensorRT для оптимизации нейросетей

В этом уроке мы рассмотрим на практике оптимизацию и инференс нейронной сети с помощью библотеки TensorRT.

Изначально TensorRT существовала в виде отдельной бибилотеки, но относительно недавно её функционал был в том числе интегрирован в TensorFlow. Именно этим мы и воспользуемся в этом уроке. При такой интеграции на выходе мы всё еще получаем TensorFlow модель, просто часть её графа будут оптимизированы и вычислены (во время инференса) с помощью TensorRT.

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


### Загрузка библиотек

In [ ]:
import numpy as np

import tensorflow as tf

### Импорт TensorRT
 Будем использовать TensorRT, котрый идёт внутри TensorFlow.

 В реальной среде может потребоваться произвести дополнительную установку недостающих компонентов TensorRT, но в Google Colab уже установлено всё, что нужно.


In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

### Создание модели 

Создадим свёрточную нейронную сеть с большим количеством свёрточных слоёв.

Кроме того, для TensorRT оптимизации необходимо фиксировать размер входа. Делаем это с помощью метода `_set_inputs()`

In [ ]:
model = tf.keras.Sequential([
     tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(32, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.MaxPool2D((2, 2)),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.MaxPool2D((2, 2)),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.Conv2D(64, (5, 5), padding='same', activation='relu'),
     tf.keras.layers.MaxPool2D((2, 2)),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dense(10, activation='softmax'),
])

model._set_inputs(np.zeros((1, 28, 28, 1), dtype=np.float32))

### Сохранение модели 

Сохраним модель в виде `saved_model`

In [ ]:
model.save('saved_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/assets


### Оптимизация модели с помощью TensorFlow

А теперь давайте произведем оптимизацию нашей модели с помощью TensorRT.

Сначала создадим TensorRT конвертер (`converter`) и укажем ему путь до нашей сохранённой неоптимизированной TensorFlow модели. Если нужно как-то еще сконфигурировать процесс оптимизации, можно передать дополнительные параметры в конструктор `TrtGraphConverterV2`, но мы будем использовать параметры по умолчанию.

После этого просто вызываем метод `convert()` и TensorRT применит все свои стратегии для оптимизации нашей модели. Важно запускать этот ноутбук в режиме GPU, так как TensorRT работает только с GPU.

После оптимизации можно сохранить новую модель в стандартном формате `saved_model`

In [ ]:
converter = trt.TrtGraphConverterV2(input_saved_model_dir='saved_model')
converter.convert()
converter.save('saved_model_trt')

INFO:tensorflow:Linked TensorRT version: (0, 0, 0)
INFO:tensorflow:Loaded TensorRT version: (0, 0, 0)
INFO:tensorflow:Assets written to: saved_model_trt/assets


### Загрузка оптимизированной TensorRT модели

Теперь можно загрузить оптимизированную модель и произвести инференс. Во время инференса такой модели TensorFlow будет обращаться к инференс движку TensorRT.

In [ ]:
model_trt = tf.keras.models.load_model('saved_model_trt')

# Warm-up
_=model_trt(np.zeros((1, 28, 28, 1), dtype=np.float32))

### Сравнение скорости работы двух моделей

Запустим инференс для обеих моделей со случайным входом и узнаем среднюю скорость работы каждой из моделей с помощью магической команды `%%timeit`

In [ ]:
%%timeit -n 10 -r 10

q = model(np.zeros((1, 28, 28, 1), dtype=np.float32))

10 loops, best of 10: 5.58 ms per loop


In [ ]:
%%timeit -n 10 -r 10

q = model_trt(np.zeros((1, 28, 28, 1), dtype=np.float32))

10 loops, best of 10: 1.07 ms per loop


TensorRT модель получилась быстрее.

**[Задание 1]** Сравните скорость работы двух моделей при различных гиперпараметрах (размер батча, размер входа, количество слоёв). Рассчитайте коэффициент ускорения (во сколько раз одна модель быстрее другой) для каждой конфигурации. Постройте соответствующие графики. (Например, график зависимости ускорения от размера входа при условии, что всё остальное фиксировано). Для этого задания вам понадобится самостоятельно реализовать способ измерения среднего времени инференса модели.